In [ ]:
from diagrams import Cluster, Diagram, Edge
from diagrams.aws import compute, database, enduser, integration, network, storage
from diagrams.generic import storage as generic_storage
from diagrams.onprem import client
from diagrams.programming import flowchart, language


show_diagrams = True

def display_diagram(diag):
    if show_diagrams:
        display(diag)

In [ ]:
graph_attr = {
    "layout": "dot",
    "compound": "true",
}

transparent_bg = {
    # "label":"x",
    "bgcolor":"transparent",
    # "penwidth":"0.0"    
}

with Diagram(filename="workspaces", outformat="png", show=False, direction="LR", graph_attr=graph_attr) as diag:
    with Cluster("AWS TRE Account", graph_attr=transparent_bg):
        with Cluster("Project A S3"):
            projecta = [
                storage.S3("Study A"),
                storage.SimpleStorageServiceS3Bucket("User-1 files"),
            ]
        with Cluster("Project B S3"):
            projectb = [
                storage.S3("Study B"),
                storage.SimpleStorageServiceS3Bucket("User-1 files"),
            ]

    with Cluster("AWS Researchers Account", graph_attr=transparent_bg):
        with Cluster("Secure compute A"):
            windows = compute.EC2Instance("Windows")
            rdp = enduser.Appstream20("RDP")
        with Cluster("Secure compute B"):
            hpc = compute.Batch("HPC")
            ubuntu = compute.EC2("Ubuntu")
            vnc = enduser.Appstream20("VNC")

    projecta[1] - windows << rdp
    [projectb[0] - hpc, projectb[1]] - ubuntu << vnc
    [rdp, vnc] << client.User("User-1")

display_diagram(diag)

In [ ]:
with Diagram(filename="networks", outformat="png", show=False, direction="TB", graph_attr=graph_attr) as diag:
    #, graph_attr=transparent_bg
    with Cluster("AWS TRE Account"):
        s3 = storage.S3("Object store")
        hictre = network.CloudFront("https://hic-tre")
        database.Dynamodb("DynamoDB") - integration.StepFunctions("orchestration\nof workspaces") << hictre

    with Cluster("Shared services"):
        services = [
            compute.EC2Instance("License server"),
            compute.EC2Instance("Anti-virus proxy"),
            compute.EC2Instance("CRAN mirror"),
        ]
        network_services = network.VPC("Services network")
        services - network_services - network.NATGateway("NAT Gateway")

    with Cluster("AWS Researchers Account"):
        network_researchers = network.VPC("No external\ninternet")

    # network_tre = network.VPC("TRE")
    # s3 - network_tre
    # researchers - network_researchers

    s3 << network.Privatelink("S3 PrivateLink") << network_researchers
    network_services - network.TransitGateway("TransitGateway") - network_researchers
    hictre << client.Users()

display_diagram(diag)

In [ ]:
with Diagram(filename="networkflexibility", outformat="png", show=False, direction="LR", graph_attr=graph_attr) as diag:
    #, graph_attr=transparent_bg
    with Cluster("AWS TRE Account"):
        hictre = network.CloudFront("https://hic-tre")

    with Cluster("Shared research AWS account"):
        gw4 = network.TransitGateway("TransitGateway")
        gw4 - network.VPC() << [
            network.PrivateSubnet("Project 4"),
            network.PrivateSubnet("Project 5"),
            network.PrivateSubnet("Project 6"),
        ]

    with Cluster("Dedicated research AWS account 1"):
        gw1 = network.TransitGateway("TransitGateway")
        gw1 - network.VPC() << network.PrivateSubnet("Project 1")

    with Cluster("Dedicated research AWS account 2"):
        gw2 = network.TransitGateway("TransitGateway")
        gw2 - network.VPC() << network.PrivateSubnet("Project 2")

    with Cluster("Dedicated research AWS account 3"):
        gw3 = network.TransitGateway("TransitGateway")
        gw3 - network.VPC() << network.PrivateSubnet("Project 3")

    gw4 >> hictre << [gw1, gw2, gw3]

display_diagram(diag)